<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_14_Traders_trade_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 13 and intro

Script Notes: 

In lesson 13 traders detrmined who were in the von neumann cells next to them, now in this lesson the agent and those agents need to determine if they should trade with each. 

*Review each class and model set up*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import math
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 4.8 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 


In [ ]:
def get_distance(pos_1, pos_2):
    """Calculate Euclidean distance between two positions.
    Args:
        pos_1, pos_2: Coordinate tuples for both points.
    
    used in trader.move()
    """
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return math.sqrt(dx**2 + dy**2)

In [ ]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      
      

In [ ]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn. (rule G1)
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
       

## Add Trade 

**Part 3**

------------

Similar to the move function were the agent needs to weigh all its options, we need to first detemrine how does an agent understand its needs and then trade with its neighbors. 

Before we determine how to trade with neighbors we first need to find all our neighbors. So again we will use list comprehension, and in this case we will need a new helper function get_trader but we can reuse our is occupied function so in this case we want traders. 

Now we build our get_trader function and for my own preference I will put with all the get functions we have already built to keep my code more organized

using mesa get cell content function will we get all the cell contents in our von nuemann neighborhood, then we iterate through this list and check for nay intances of a Trade, knowing there can only be one in a cell and we return that Trade object if there is, we now check to see if we are getting a list of traders. So I will add a print statement, change the number of steps back to one and sure enough we are getting a list of agent objects. AS you can see some agents have no agents in the area so if that is the case we can just reutrn an empty list to make the code more efficient. The question of course is why am I return anything. As this is fundamentally an economics model, we want to get all the prices for trade so we can ensure it matches economic theory as validation for our model. So we want to collect the price data which will put into our data collector later on for analysis. 

so we need to add an attribute to our agent which will be an empty list called prices and then we will return the list of prices from the trade_with_neighbors function. 

Please appreciate we built and refined this model previously, that invovled a lot of errors and optimizations to make the code more efficient. In short we know what decisions to make as we already have a working model, but this does not reflect the reality of buidling a model from scratch.  


In [ ]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 
      self.prices = []

  
    def get_sugar(self, pos): #part 2
      '''
      used in:
         get_sugar_amount()
         eat()

      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Sugar:
              return agent
      return None
    
    
    def get_sugar_amount(self, pos):
      '''
      used in move()
      '''
      
      sugar_patch = self.get_sugar(pos)
      if sugar_patch:
          return sugar_patch.amount
      return 0
    
    
    def get_spice(self, pos):
      '''
      used in get_spice_amount
      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Spice:
              return agent
      return None
        
    
    def get_spice_amount(self, pos):
        '''
        used in move()
        '''

        spice_patch = self.get_spice(pos)
        if spice_patch: 
          return spice_patch.amount
        return 0
  
    #part 3
    def get_trader(self, pos):
      '''
      used in trade_with neighbors()
      '''
      this_cell =self.model.grid.get_cell_list_contents(pos)

      for agent in this_cell: 
        if isinstance(agent, Trader):
          return agent
    
    
    def is_occupied(self, pos): 
      '''
      Helper function for move()
      '''
      
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell:
          if isinstance(a, Trader) and a.pos != self.pos:
              return True
      return False
       
    def calculate_welfare(self, sugar, spice): #This will be built to be like the exmaple later with none, but right now it wouldn't make any sense
      '''
      Helper function for move
      '''

      #calculate total resources
      m_total = self.metabolism_sugar + self.metabolism_spice
      #Cobb-Douglas functional form
      return sugar** (self.metabolism_sugar/m_total) * spice ** (
          self.metabolism_spice/m_total)
      
    def is_starved(self): 
      '''
      helpfer function for maybe_die()
      '''

      return (self.sugar <= 0) or (self.spice <=0)
    
    ###########################################################################
    #                                                                         #
    #                            MAIN FUNCTIONS                               #
    #                                                                         #
    ###########################################################################

    def move(self): 
           
      # 1. Get neighbors within vision

      neighbors = [i 
                   for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore,True, self.vision
                       )
                   if not self.is_occupied(i)
      ]

      # 2. Find the patch which porduce the maximum welfare. 

      welfares = [
            self.calculate_welfare(
                self.sugar + self.get_sugar_amount(pos),
                self.spice + self.get_spice_amount(pos),
            )
            for pos in neighbors] #part 2

      #find the highest welfare in the cell 
      max_welfare = max(welfares)
      #get the index of maximal welfare cells
      candidate_indices = [i for i in range(len(welfares))
                             if math.isclose(welfares[i], max_welfare,
                                             rel_tol=1e-02)]
      #convert index to positions of those cells
      candidates = [neighbors[i] for i in candidate_indices]
     
      
      # 3. Find the nearest patch among the candidate.
      min_dist = min([get_distance(self.pos, pos) for pos in candidates])
      final_candidates = [
          pos
          for pos in candidates
          if math.isclose(get_distance(self.pos, pos),min_dist, rel_tol=1e-02)
      ]
      self.random.shuffle(final_candidates)

      # 4. Move agent.
      self.model.grid.move_agent(self, final_candidates[0])



    def eat(self):
      #get sugar  #part 1
      #print(self.sugar)
      sugar_patch = self.get_sugar(self.pos)
      #print(sugar_patch)
      if sugar_patch:
        self.sugar = self.sugar - self.metabolism_sugar + sugar_patch.amount
        sugar_patch.amount = 0
        #print(self.sugar, sugar_patch.amount)
      
      #get_spice part 2
      spice_patch = self.get_spice(self.pos)
      #print(spice_patch)
      if spice_patch: 
        self.spice = self.spice - self.metabolism_spice + spice_patch.amount
        spice_patch.amout = 0
        #print(self.spice, spice_patch.amount)

    def maybe_die(self): 

      if self.is_starved(): 
        print(self.unique_id, self.model.schedule.get_type_count(Trader))
        self.model.grid.remove_agent(self) 
        self.model.schedule.remove(self)
        print(self.unique_id, self.model.schedule.get_type_count(Trader))


    #Part 3
    def trade_with_neighbors(self):
      
      neighbor_agents =[self.get_trader(pos) for pos in self.model.grid.get_neighborhood(
          self.pos, self.moore, False, radius=self.vision) if self.is_occupied(pos)]
        
      if len(neighbor_agents) == 0:
            return []
      



# Part 1

--------------------------

As discussed in lesson 3, one reason we picked this model was to explore more deeply Agent Based Models combined with the dynamics of Python. So the first step we need to do is not necessarily intuitive. If we look at our step function we realize we are not using Mesa's dictionary of our agents, we are using a seperate list and we just removed agents so now we have a discrepancy that will give us an error. To see this we can create another loop and ask the agents to move. We then get a type error that we cannot unpack a non-iterable, whichin this case is None. This because our Trader shuffle has agents that have since been removed from out agent dictionary, so we need to repreat our creation of the Trader_shuffle list. 

# Part 2

--------------------------

As we want to make sure we are using the DRY or Do Not Repeat yourself principle we will put this in a helper function. We can now run the model with no issues  

Now we can iterate through the list and call for agents to trade, so we will create a function called trade with neighbors that we will call from the trade class so now start buidling that function 




In [ ]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min = 1,
                 metabolism_max = 5, vision_min = 1, vision_max =5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population 
    self.endowment_min = endowment_min 
    self.endowment_max = endowment_max 
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

    sugar_distribution = np.genfromtxt("/content/drive/MyDrive/sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) 
      y = self.random.randrange(self.height) 
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.metabolism_min, self.metabolism_max)
      metabolism_spice = self.random.uniform(self.metabolism_min, self.metabolism_max)
      #add vision
      vision =int(self.random.uniform(self.vision_min, self.vision_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  
  #part 2
  def randomize_traders(self): 
    """
    Helper function for step()

    updates list of agents used in step function
    """
    
    Traders = self.schedule.agents_by_type[Trader].values()
    Trader_shuffle = list(Traders)
    self.random.shuffle(Trader_shuffle)

    return Trader_shuffle
  
  
  def step(self): 
    
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()
        
    #part t
    Trader_shuffle = self.randomize_traders()
    
    for agent in Trader_shuffle: 
      agent.move() 
      agent.eat() 
      agent.maybe_die() 

    '''
    #demo error part 1
    for agent in Trader_shuffle:
      agent.move()
    '''
    #part 2
    Trader_shuffle = self.randomize_traders()
    

    #part 2
    for agent in Trader_shuffle: 
      #agent.trade_with_neighbors()
      agent.prices = agent.trade_with_neighbors()
    

    self.schedule.time +=1
    self.schedule.steps += 1
    

  
  def run_model(self, step_count=1000):
    
    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

## Model Run 


In [ ]:
model = SugarscapeG1mt()
model.run_model(step_count=1) #part 3 change to 1

